In [47]:
from dataclasses import dataclass
from enum import Enum, auto
from typing import Self
from uuid import uuid4
from bamt.preprocessors import Preprocessor
import pandas as pd
from sklearn import preprocessing as pp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import re


In [48]:
df = pd.read_csv("/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/mig whereabouts/inflow LO.csv")

Выделим ОКТМО из Ленобласти

In [49]:
oktmos = map(str,df.oktmo.unique())

In [50]:
oktmos_lo = [x for x in oktmos if re.fullmatch(r"41\d+", x)]

In [51]:
del oktmos

In [52]:
len(oktmos_lo)

226

Определим численность населения. Проблема: удалось найти данные только за 2024 год

In [53]:
pop_size_df = pd.read_excel("/Users/pishchulov/edu/matobes/НИР/outer_source/численность МО/Сhisl_MO_01-01-2024_only_LO.xlsx")

In [54]:
pop_size_df

,Коды территорий,Unnamed: 1,население,городское,сельское
0,4100000000,Ленинградская область,2035762,1373533,662229
1,41754000 0 0,Сосновоборский городской округ,63462,63462,0
2,41754000001 1 0 0 0,г Сосновый Бор,63462,63462,0
3,41603000 0 0,Бокситогорский муниципальный район,50855,39107,11748
4,41603101 0 0,Городское поселение Бокситогорское,15764,15480,284
...,...,...,...,...,...
271,41648418 0 0,Сельское поселение Нурминское,3338,0,3338
272,41648430 0 0,Сельское поселение Лисинское,1875,0,1875
273,41648443 0 0,Сельское поселение Тельмановское,26781,0,26781
274,41648444 0 0,Сельское поселение Трубникоборское,1599,0,1599


In [55]:
def discard_to_8_chars(s):
    if len(s)!=10:
        return s
    if s[-2:]=="00":
        return s[:-2]        

In [56]:
pop_size_df["Коды территорий"] = pd.Series([discard_to_8_chars(x.replace(" ","")) for x in map(str, pop_size_df["Коды территорий"])], dtype=str)

In [57]:
# pop_size_df_lo = pop_size_df[pop_size_df["Коды территорий"].isin(oktmos_lo)]


In [58]:
# set(oktmos_lo) - set(pop_size_df_lo["Коды территорий"])

Проблема: некоторые ОКТМО исчезли, потому что они эти населенные пункту переехали на новые коды. Проигнорируем.

In [59]:
del pop_size_df

In [60]:
# pop_size_df_lo=pop_size_df_lo.drop(["городское", "сельское"], axis = 1)
# pop_size_df_lo=pop_size_df_lo.set_axis(["oktmo", "name", "pop_size"], axis=1)
# pop_size_df_lo.sort_values("oktmo", inplace=True)
# pop_size_df_lo

Удалим муниципальные районы, потому что это агрегаты других МО

In [61]:
def is_mun_district(oktmo):
    assert len(oktmo)==8
    return oktmo[-3:]=="000"

In [62]:
# pop_size_df_lo.pop_size.sum()

In [63]:
# mun_districts_lo = pop_size_df_lo[pop_size_df_lo.oktmo.apply(is_mun_district)]
# mun_districts_lo

In [64]:
# pop_size_df_lo=pop_size_df_lo[~pop_size_df_lo.oktmo.apply(is_mun_district)]
# pop_size_df_lo.pop_size.sum()

### Подготовим данные о среде. 

In [65]:
allmun = pd.read_csv("/Users/pishchulov/edu/matobes/НИР/outer_source/rosstat_allmun/allmuns/popsize (allmun).csv")
allmun[["oktmo"]] = allmun[["oktmo"]].astype(str)
allmun["from_lo"] = [bool(re.fullmatch(r"41\d+", x)) for x in allmun.oktmo]
allmun = allmun[(allmun.from_lo) & (allmun.year == 2022)]
allmun = allmun[["oktmo", "name", "year"]].set_index("oktmo")

In [66]:
allmun

,name,year
oktmo,,
41603000,Бокситогорский муниципальный район,2022
41603101,Бокситогорское,2022
41603102,Пикалёвское,2022
41603155,Ефимовское,2022
41603412,Большедворское,2022
...,...,...
41648430,Лисинское,2022
41648443,Тельмановское,2022
41648444,Трубникоборское,2022


Нет ITMO2 данных за 2023 год, поэтому используем 2022.

In [67]:

mo_features = allmun
# files = [
#     "shoparea",
#     "foodseats",
#     "agrprod",
#     "beforeschool",
#     "schoolnum",  # тут нет требуемых районов
#     "museums",
#     "theatres",
#     "musartschool",
#     "hospitalcap",
#     "pollutionvol",
#     "popsize",
#     "retailturnover",
#     "livarea",

# ]
files = """parks 
schoolnum 
cliniccap 
sportsvenue 
consnewareas 
popsize 
livestock 
saldo internat 
consnewapt 
budincome 
musartschool 
foodservturnover 
goodcompanies 
invest 
factoriescap 
shoparea 
museums 
agrprod 
roadslen 
hospitals 
outflow 
foodseats 
servicesnum 
avgsalary 
harvest 
funds 
library 
cultureorg 
saldo interreg 
avgemployers 
naturesecure 
goodcompincome 
visiblecompanies 
retailturnover 
beforeschool 
theatres 
saldo reg 
docsnum 
livarea 
badcompanies """.split('\n')

files_about_mo = ["sportsvenue", "invest", "popsize", "shoparea", "roadslen", "hospitals"]
for f in files_about_mo:
    d = pd.read_csv(f"/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/superdataset/features separately/{f} (allmun).csv")
    d[["oktmo"]] = d[["oktmo"]].astype(str)
    d["from_lo"] = [bool(re.fullmatch(r"41\d+", x)) for x in d.oktmo]
    # d_lo = d[(d.from_lo) & (d.year == 2022)]
    # print(f"{f} {len(d_lo)}")
    d = d[(d.oktmo.isin(mo_features.index))&(d.year == 2022)].drop(["from_lo"], axis=1)
    if len(d)==0:
        print(f"error in {f}")
    else:
        mo_features = mo_features.join(d.set_index("oktmo").iloc[:,-1:], )

In [68]:
mo_features.head(20)

,name,year,sportsvenue,invest,popsize,shoparea,roadslen,hospitals
oktmo,,,,,,,,
41603000,Бокситогорский муниципальный район,2022,167.0,1173969.0,47236.0,39624.7,329.4,11.0
41603101,Бокситогорское,2022,47.0,460738.0,14606.0,17127.5,56.0,1.0
41603102,Пикалёвское,2022,62.0,632704.0,19250.0,18216.3,47.0,1.0
41603155,Ефимовское,2022,19.0,35548.0,4389.0,2255.1,136.4,3.0
41603412,Большедворское,2022,8.0,2752.0,1486.0,396.0,28.2,1.0
41603416,Борское,2022,15.0,16573.0,3272.0,775.0,24.3,2.0
41603460,Лидское,2022,6.0,12516.0,2137.0,520.5,77.8,2.0
41603476,Самойловское,2022,10.0,13138.0,2096.0,334.3,52.8,1.0
41606000,Волосовский муниципальный район,2022,117.0,2783004.0,51600.0,40372.9,100.1,32.0


In [69]:
len(mo_features)

205